In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
import re
import string
import os,glob
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import keras
from nltk.corpus import stopwords

In [ ]:
import pandas as pd
excel_file = r'/content/drive/MyDrive/Embeddings/Fully compiled dataset Sentence New.xlsx'
df = pd.read_excel(excel_file)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
df['Type'] = df['Type'].replace(mapping)
df_type_0 = df[df['Type'] == 0]
df_type_1 = df[df['Type'] == 1]

# Shuffle both DataFrames
df_type_0 = df_type_0.sample(frac=1).reset_index(drop=True)
df_type_1 = df_type_1.sample(frac=1).reset_index(drop=True)

# Calculate the length of the longest DataFrame
max_length = max(len(df_type_0), len(df_type_1))

# Create a new DataFrame to hold the alternating rows
randomized_df = pd.DataFrame(columns=df.columns)

# Iterate through both DataFrames and append rows alternately until one of them runs out
for i in range(max_length):
    if i < len(df_type_0):
        randomized_df = randomized_df.append(df_type_0.iloc[i], ignore_index=True)
    if i < len(df_type_1):
        randomized_df = randomized_df.append(df_type_1.iloc[i], ignore_index=True)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
randomized_df['Type'] = randomized_df['Type'].replace(mapping)
randomized_df.drop('ID', axis=1, inplace=True)
randomized_df

<ipython-input-3-5e724572395f>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  randomized_df = randomized_df.append(df_type_0.iloc[i], ignore_index=True)
<ipython-input-3-5e724572395f>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  randomized_df = randomized_df.append(df_type_1.iloc[i], ignore_index=True)


,Text,Type
0,ورزش کرنے والے افراد چست اورچاک وچوبند رہتے ہی...,0
1,کرکٹ، کبڈی، فٹ بال وغیرہ جیسے کھیل ہماری جسمان...,1
2,قائد اعظم محمد علی جناح نے مختلف تقاریر کے ان ...,0
3,محنت کرنے والے سے اللہ پاک بھی خوش ہوتے ہیں۔ ا...,1
4,ابتدائی تعلیم کے بعد قائداعظم اعلی تعلیم حاصل ...,0
...,...,...
2515,۔ یہ ٹیکنالوجی دنیا کے اربوں انسانوں کو جدید ا...,1
2516,دوسروں کو جگر ڈونیٹ کیا جا سکتا ہے ۔یہ قدرت کے...,0
2517,مطالعہ کتب، بحث و مباحثہ کے ساتھ ساتھ ان کی اخ...,1
2518,1937ء میں وہ آل انڈیا مسلم لیگ کے صدر منتخب ہ...,0


In [ ]:
import pandas as pd
excel_file = r'/content/drive/MyDrive/Embeddings/Fully compiled dataset Sentence New.xlsx'
df = pd.read_excel(excel_file)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
df['Type'] = df['Type'].replace(mapping)
df_type_0 = df[df['Type'] == 0]
df_type_1 = df[df['Type'] == 1]

# Shuffle both DataFrames
df_type_0 = df_type_0.sample(frac=1).reset_index(drop=True)
df_type_1 = df_type_1.sample(frac=1).reset_index(drop=True)

# Calculate the length of the longest DataFrame
max_length = max(len(df_type_0), len(df_type_1))

# Create a new DataFrame to hold the alternating rows
randomized_df = pd.DataFrame(columns=df.columns)

# Iterate through both DataFrames and append rows alternately until one of them runs out
for i in range(max_length):
    if i < len(df_type_0):
        randomized_df = randomized_df.append(df_type_0.iloc[i], ignore_index=True)
    if i < len(df_type_1):
        randomized_df = randomized_df.append(df_type_1.iloc[i], ignore_index=True)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
randomized_df['Type'] = randomized_df['Type'].replace(mapping)
randomized_df.drop('ID', axis=1, inplace=True)
randomized_df

<ipython-input-4-5e724572395f>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  randomized_df = randomized_df.append(df_type_0.iloc[i], ignore_index=True)
<ipython-input-4-5e724572395f>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  randomized_df = randomized_df.append(df_type_1.iloc[i], ignore_index=True)


,Text,Type
0,جسمانی طور پہ چاق و چوبند ہونے کی اہمیت کے با...,0
1,تعلیم کی اہمیت ازل سے ابد تک رہے گی خواہ وہ مذ...,1
2,آخر میں اسی اہم مسئلہ کی طرف میں اہل علم کی تو...,0
3,مذکورہ بالا کورسیس کے علاوہ بھی کمپیوٹر کے مخت...,1
4,یہ ایک مسلمہ حقیقت ہے کہ سرسید کو خدا نے ایک خ...,0
...,...,...
2515,۔ ہر دن کوئی نہ کوئی نئی ایجاد اور نئے طور طری...,1
2516,آپ کا قد پانچ فٹ ساڑھے گیارہ (11)انچ تھا۔ رنگ...,0
2517,جنگلات کی تعریف میں ہم دیکھتے ہیں کہ یہ وہ سرگ...,1
2518,علی گڑھ تحریک کو ترقی دینے میں سرسید کے اہم م...,0


In [ ]:
randomized_df['Text'] = randomized_df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split())) #convert text into lower case

randomized_df['Text'] = randomized_df['Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])    # Remove URLs

#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x[0]!="@"))

#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x[0]!="#"))

#dataset_df['Text'] = dataset_df['Text'].str.replace('\d+', '')  #Remove digits

randomized_df['Text'] = randomized_df['Text'].map(lambda x: re.sub(r'\W+', ' ', x)) # Remove Punctuation marks

#stop = stopwords.words('english')
#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) #Remove stop words

print("\nDataset After Text Preprocessing:")
print("=================================================\n")
randomized_df.head(7)


Dataset After Text Preprocessing:



,Text,Type
0,جسمانی طور پہ چاق و چوبند ہونے کی اہمیت کے بار...,0
1,تعلیم کی اہمیت ازل سے ابد تک رہے گی خواہ وہ مذ...,1
2,آخر میں اسی اہم مسئلہ کی طرف میں اہل علم کی تو...,0
3,مذکورہ بالا کورسیس کے علاوہ بھی کمپیوٹر کے مخت...,1
4,یہ ایک مسلمہ حقیقت ہے کہ سرسید کو خدا نے ایک خ...,0
5,محنت کی اہمیت لامحدود ہے محنتی آدمی ناممکن کو ...,1
6,پروفیسر صاحب جدیدترین ا نکھوں کے کالج کے پرنسپ...,0


In [ ]:
!pip install --upgrade gensim -q
!pip install matplotlib -q

In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string

In [ ]:
np.random.seed(42)

In [ ]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load('/content/drive/MyDrive/Embeddings/word2vec-google-news-300.kv')

In [ ]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [ ]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
sent_vec("I am happy")

In [ ]:
randomized_df['vec'] = randomized_df['Text'].apply(sent_vec)

In [ ]:
randomized_df.head()

In [ ]:
X = randomized_df['vec'].to_list()
y = randomized_df['Type'].to_list()

In [ ]:
y

In [ ]:
X = np.stack(randomized_df['vec'].values)

In [ ]:
X = X.reshape(X.shape[0], 1, X.shape[1])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from keras.layers import GRU, Embedding, Dense, Dropout
from keras.models import Sequential
from keras import optimizers

In [ ]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
y_test

In [ ]:
# Define the Single-Layer GRU Model
GRU_model = Sequential()
GRU_model.add(GRU(50, input_shape=(X.shape[1], X.shape[2]), recurrent_dropout=0.2))
GRU_model.add(Dense(25, activation='relu'))
GRU_model.add(Dropout(0.2))
GRU_model.add(Dense(12, activation='relu'))
GRU_model.add(Dropout(0.2))
GRU_model.add(Dense(1, activation='sigmoid'))  # Change output units to 1 for binary classification


In [ ]:
# Compile the GRU Model
adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
GRU_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', 'Precision', 'Recall'])

In [ ]:
# Convert X_train and y_train to NumPy arrays if they are not already
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
(X_train)

In [ ]:
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
# Train the GRU Model
model_report = GRU_model.fit(X_train, y_train, epochs=30, batch_size=batch_size, validation_split=0.2)


Epoch 1/30
2/2 [==============================] - 0s 167ms/step - loss: 0.0000e+00 - accuracy: 0.5087 - precision_1: 0.4516 - recall_1: 0.0177 - val_loss: 0.0000e+00 - val_accuracy: 0.4752 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/30
2/2 [==============================] - 0s 131ms/step - loss: 0.0000e+00 - accuracy: 0.5136 - precision_1: 0.7778 - recall_1: 0.0089 - val_loss: 0.0000e+00 - val_accuracy: 0.4752 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/30
2/2 [==============================] - 0s 159ms/step - loss: 0.0000e+00 - accuracy: 0.5081 - precision_1: 0.1667 - recall_1: 0.0013 - val_loss: 0.0000e+00 - val_accuracy: 0.4752 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/30
2/2 [==============================] - 0s 128ms/step - loss: 0.0000e+00 - accuracy: 0.5093 - precision_1: 0.3333 - recall_1: 0.0025 - val_loss: 0.0000e+00 - val_accuracy: 0.4752 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 5/30
2/2 [

In [ ]:
# Predict the probabilities for each class for test data
y_pred_probs = GRU_model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate accuracy, precision, recall, and F1 score on testing data
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Test Accuracy: {accuracy}')
print(f'Test Precision: {precision}')
print(f'Test Recall: {recall}')
print(f'Test F1 Score: {f1}')


16/16 [==============================] - 0s 19ms/step
Test Accuracy: 0.4861111111111111
Test Precision: 0.24305555555555555
Test Recall: 0.5
Test F1 Score: 0.32710280373831774


In [ ]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.models import Sequential

In [ ]:
# Define the CNN model without pooling
CNN_model = Sequential()
CNN_model.add(Conv1D(24, 5, padding='same', activation='relu', strides=1, input_shape=(X.shape[1], X.shape[2])))
CNN_model.add(Flatten())
CNN_model.add(Dense(12, activation='relu'))
CNN_model.add(Dropout(0.2))
CNN_model.add(Dense(6, activation='relu'))
CNN_model.add(Dropout(0.2))
CNN_model.add(Dense(1, activation='sigmoid'))  # Change output units to 1 for binary classification

In [ ]:
# Compile the CNN Model
adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
CNN_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', Precision(), Recall()])

In [ ]:
# Train the CNN Model
batch_size = 1500
model_report = CNN_model.fit(X_train, y_train, epochs=30, batch_size=batch_size, validation_split=0.2)

Epoch 1/30
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - accuracy: 0.4876 - precision_3: 0.4866 - recall_3: 0.8517 - val_loss: 0.0000e+00 - val_accuracy: 0.4604 - val_precision_3: 0.1250 - val_recall_3: 0.0047
Epoch 2/30
2/2 [==============================] - 0s 120ms/step - loss: 0.0000e+00 - accuracy: 0.5012 - precision_3: 0.4807 - recall_3: 0.2370 - val_loss: 0.0000e+00 - val_accuracy: 0.4728 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 3/30
2/2 [==============================] - 0s 123ms/step - loss: 0.0000e+00 - accuracy: 0.4882 - precision_3: 0.4072 - recall_3: 0.1001 - val_loss: 0.0000e+00 - val_accuracy: 0.4752 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 4/30
2/2 [==============================] - 0s 125ms/step - loss: 0.0000e+00 - accuracy: 0.5136 - precision_3: 0.5287 - recall_3: 0.0583 - val_loss: 0.0000e+00 - val_accuracy: 0.4752 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 5/30
2/2 [========

In [ ]:
!pip install tensorflow-text

In [ ]:
# Transformers installation
! pip install transformers
# To install from source instead of the last release, comment the command above and uncomment the following one.
! pip install git+https://github.com/huggingface/transformers.git


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-6rklf79u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-6rklf79u
  Resolved https://github.com/huggingface/transformers.git to commit 838b87abe231fd70be5132088d0dee72a7bb8d62
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8756970 sha256=420fd009db004b58a6c99e62a5adf1188276cba38c9f89223a70f85a396e204a
  Stored in directory: /tmp/pip-ephem-wheel-cache-wu8zxiyk/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")


NameError: name 'hub' is not defined

In [ ]:
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")